## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline to Resolve Medication Codes


- **Model**: `en.resolve.medication`
- **Model Description**: This pretrained resolver pipeline extracts medications and resolves their adverse reactions (ADE), RxNorm, UMLS, NDC, and SNOMED CT codes, as well as action/treatments, from clinical text, providing support for both branded and non-branded medications.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [19]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [20]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [21]:
model_name = "en-resolve-medication"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [23]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

INFO:sagemaker:Creating model with name: en-resolve-medication-2024-05-17-07-44-20-320
INFO:sagemaker:Creating endpoint-config with name en-resolve-medication
INFO:sagemaker:Creating endpoint with name en-resolve-medication


--------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [24]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [25]:
docs = [
    "The patient was given Adapin 10 MG, coumadin 5mg",
    "She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",
    "The doctor prescribed Lisinopril 20 MG once daily for hypertension."
]



sample_text = """The patient was prescribed Amlodopine Vallarta 10-320mg, Eviplera. The other patient is given Lescol 40 MG and Everolimus 1.5 mg tablet."""


### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [26]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,ade,rxnorm_code,action,treatment,umls_code,snomed_ct_code,product_ndc,package_ndc
0,"The patient was prescribed Amlodopine Vallarta 10-320mg, Eviplera. The other patient is given Lescol 40 MG and Everolimus 1.5 mg tablet.",Amlodopine Vallarta 10-320mg,27,54,DRUG,0.8083666,Gynaecomastia,722131,NONE,NONE,C1949334,425838008,00093-7693,00093-7693-56
1,"The patient was prescribed Amlodopine Vallarta 10-320mg, Eviplera. The other patient is given Lescol 40 MG and Everolimus 1.5 mg tablet.",Eviplera,57,64,DRUG,None,Anxiety,217010,Inhibitory Bone Resorption,Osteoporosis,C0720318,NONE,NONE,NONE
2,"The patient was prescribed Amlodopine Vallarta 10-320mg, Eviplera. The other patient is given Lescol 40 MG and Everolimus 1.5 mg tablet.",Lescol 40 MG,94,105,DRUG,0.60043335,NONE,103919,Hypocholesterolemic,Heterozygous Familial Hypercholesterolemia,C0353573,NONE,00078-0234,00078-0234-05
3,"The patient was prescribed Amlodopine Vallarta 10-320mg, Eviplera. The other patient is given Lescol 40 MG and Everolimus 1.5 mg tablet.",Everolimus 1.5 mg tablet,111,134,DRUG,0.719375,Acute myocardial infarction,2056895,NONE,NONE,C4723581,NONE,00054-0604,00054-0604-21


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [27]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,ade,rxnorm_code,action,treatment,umls_code,snomed_ct_code,product_ndc,package_ndc
0,"The patient was given Adapin 10 MG, coumadin 5mg",Adapin 10 MG,22,33,DRUG,0.6684,NONE,1000049,NONE,NONE,NONE,NONE,NONE,NONE
1,"The patient was given Adapin 10 MG, coumadin 5mg",coumadin 5mg,36,47,DRUG,0.6889,NONE,855333,Anticoagulant,Cerebrovascular Accident,C2709920,NONE,NONE,NONE
2,"She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",folic acid 1 mg,13,27,DRUG,0.67852503,Dyspnoea,315966,NONE,NONE,C0986175,NONE,11528-0135,11528-0135-01
3,"She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",levothyroxine 0.1 mg,37,56,DRUG,0.8457667,Dyspnoea,1115267,NONE,NONE,C3162905,NONE,42023-0201,42023-0201-01
4,"She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",aspirin 81 mg,62,74,DRUG,0.72886664,Pyrexia,434451,NONE,NONE,C1363721,NONE,NONE,NONE
5,The doctor prescribed Lisinopril 20 MG once daily for hypertension.,Lisinopril 20 MG,22,37,DRUG,0.6994,NONE,314077,NONE,NONE,C0981556,318860005,00143-1268,00143-9713-01


### JSON Lines

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [28]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

In [29]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "Adapin 10 MG", "begin": 22, "end": 33, "ner_label": "DRUG", "ner_confidence": "0.6684", "ade": "NONE", "rxnorm_code": "1000049", "action": "NONE", "treatment": "NONE", "umls_code": "NONE", "snomed_ct_code": "NONE", "product_ndc": "NONE", "package_ndc": "NONE"}, {"ner_chunk": "coumadin 5mg", "begin": 36, "end": 47, "ner_label": "DRUG", "ner_confidence": "0.6889", "ade": "NONE", "rxnorm_code": "855333", "action": "Anticoagulant", "treatment": "Cerebrovascular Accident", "umls_code": "C2709920", "snomed_ct_code": "NONE", "product_ndc": "NONE", "package_ndc": "NONE"}]}
{"predictions": [{"ner_chunk": "folic acid 1 mg", "begin": 13, "end": 27, "ner_label": "DRUG", "ner_confidence": "0.67852503", "ade": "Dyspnoea", "rxnorm_code": "315966", "action": "NONE", "treatment": "NONE", "umls_code": "C0986175", "snomed_ct_code": "NONE", "product_ndc": "11528-0135", "package_ndc": "11528-0135-01"}, {"ner_chunk": "levothyroxine 0.1 mg", "begin": 37, "end": 56, "ner_label"

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [30]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

INFO:sagemaker:Deleting endpoint with name: en-resolve-medication
INFO:sagemaker:Deleting endpoint configuration with name: en-resolve-medication


## 3. Batch inference

In [31]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [32]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [34]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [35]:
process_s3_json_output_and_save(validation_json_file_name)

,document,ner_chunk,begin,end,ner_label,ner_confidence,ade,rxnorm_code,action,treatment,umls_code,snomed_ct_code,product_ndc,package_ndc
0,"The patient was given Adapin 10 MG, coumadin 5mg",Adapin 10 MG,22,33,DRUG,0.6684,NONE,1000049,NONE,NONE,NONE,NONE,NONE,NONE
1,"The patient was given Adapin 10 MG, coumadin 5mg",coumadin 5mg,36,47,DRUG,0.6889,NONE,855333,Anticoagulant,Cerebrovascular Accident,C2709920,NONE,NONE,NONE
2,"She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",folic acid 1 mg,13,27,DRUG,0.67852503,Dyspnoea,315966,NONE,NONE,C0986175,NONE,11528-0135,11528-0135-01
3,"She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",levothyroxine 0.1 mg,37,56,DRUG,0.8457667,Dyspnoea,1115267,NONE,NONE,C3162905,NONE,42023-0201,42023-0201-01
4,"She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",aspirin 81 mg,62,74,DRUG,0.72886664,Pyrexia,434451,NONE,NONE,C1363721,NONE,NONE,NONE
5,The doctor prescribed Lisinopril 20 MG once daily for hypertension.,Lisinopril 20 MG,22,37,DRUG,0.6994,NONE,314077,NONE,NONE,C0981556,318860005,00143-1268,00143-9713-01


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [37]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [38]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "Adapin 10 MG", "begin": 22, "end": 33, "ner_label": "DRUG", "ner_confidence": "0.6684", "ade": "NONE", "rxnorm_code": "1000049", "action": "NONE", "treatment": "NONE", "umls_code": "NONE", "snomed_ct_code": "NONE", "product_ndc": "NONE", "package_ndc": "NONE"}, {"ner_chunk": "coumadin 5mg", "begin": 36, "end": 47, "ner_label": "DRUG", "ner_confidence": "0.6889", "ade": "NONE", "rxnorm_code": "855333", "action": "Anticoagulant", "treatment": "Cerebrovascular Accident", "umls_code": "C2709920", "snomed_ct_code": "NONE", "product_ndc": "NONE", "package_ndc": "NONE"}]}
{"predictions": [{"ner_chunk": "folic acid 1 mg", "begin": 13, "end": 27, "ner_label": "DRUG", "ner_confidence": "0.67852503", "ade": "Dyspnoea", "rxnorm_code": "315966", "action": "NONE", "treatment": "NONE", "umls_code": "C0986175", "snomed_ct_code": "NONE", "product_ndc": "11528-0135", "package_ndc": "11528-0135-01"}, {"ner_chunk": "levothyroxine 0.1 mg", "begin": 37, "end": 56, "ner_label"

In [39]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-resolve-medication-2024-05-17-08-03-47-873


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

